In [1]:
pip install gpyopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 845.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.4/959.4 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gpyopt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83591 sha256=d51073de475a296f444969f49ae0bc0e3e9911ade8212d565c8622b8f560702a
  Stored in directory: /root/.cache/pip/wheels/72/17/52/9d818b4c60f733bf49d5cf82bc2758ebbdc57a0471137c37be
  Created wheel for GPy: filename=GPy-1.10.0-cp310-cp310-linux_x86_64.whl size=3211221 sha256=51709e320323763cb6d586d818dc85dfdb3e169ff01a832616f4ceeb257c843e
  Stored in directory: /root/.cache/pip/wheels/27/bd/9f/82ab4216eae088cba864ca0dc1d75699bd4bf6823790fb2f77
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102548 sha256=54e90edba89bdd01250d0eddd3

In [8]:
import GPy
import GPyOpt

print("GPy version:", GPy.__version__)
print("GPyOpt version:", GPyOpt.__version__)


GPy version: 1.10.0
GPyOpt version: 1.2.6


In [2]:
import GPyOpt #importing library
import numpy as np #importing numpy

Code for black-box optimization. Our goal is to maximise the output given the set of input data. The code takes in input data and use it to suggest the next point where we should evaluate our function. After we run the experiments at the parameters suggested by the model, we can append that to our existing data and then rerun the model to ask for the next point where we should be evaluating. We would have to repeat these steps until our convergence criteria.

In [3]:
#defining the domain
# --> since number of passes can take only integer values, we define it as a discrete variable that can take 25 discerete
# values i.e., the integers in the range 1 to 50

# --> similar to number of passes, the resolution we have available for speed (in mm/s) is 0.5,
# so we define this also as a discrete variable that can take 31 distinct values in the range 0.5 to 15.5
# varying at an interval of 0.5

# --> SOD is also a discrete variable that takes two values: 6.5 and 13
# (or we have the freedom, should I also increase the number of values for it)

domain =[{'name': '#Passes', 'type': 'discrete', 'domain': (1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                         10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                                                         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                                                            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50)},
        {'name': 'Speed', 'type': 'discrete', 'domain': (0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5,
                                                        5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10,
                                                        10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5)},
        {'name': 'SOD', 'type': 'discrete', 'domain': (6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13)}] #0.5 reso

In [15]:
# giving the initial data, 3 dimensional input and 1-D output
# Note: I haven't inlcuded the pristine sample in the input since
# it is a bit confusing to define the value of input parameters for it
X_init = np.array([[2, 5, 13],
                  [20, 15, 6.5],
                  [2, 5, 6.5],
                  [2, 15, 13],
                  [10, 5, 13],
                  [10, 15, 6.5],
                  [10, 5, 6.5],
                  [10, 15, 13],
                  [20, 5, 13],
                  [2, 15, 6.5],
                  [20, 5, 6.5],
                  [20, 15, 13]])
Y_init = np.array([[91.853],
                  [90.393],
                  [93.208],
                  [91.742],
                  [95.125],
                  [95.185],
                  [94.079],
                  [93.418],
                  [92.761],
                  [90.722],
                  [94.974],
                  [92.296]])

In [19]:
np.random.seed(42)
X_updating = X_init
Y_updating = Y_init
bo_step = GPyOpt.methods.BayesianOptimization(f = None, domain = domain, model_type='InputWarpedGP',
                                                  X = X_updating, Y = Y_updating, initial_design_numdata=12,
                                                  acquisition_type='EI', exact_feval=False, maximize=True)

x_next_eval = bo_step.suggest_next_locations() #Gpy-opt method to suggest next point based
print(x_next_eval)

AttributeError: ignored

In [ ]:
# X_updating = X_init
# Y_updating = Y_init

# bo_step = GPyOpt.methods.BayesianOptimization(f = None, domain = domain, model_type='GP',
#                                                   X = X_updating, Y = Y_updating, initial_design_numdata = 12,
#                                                   acquisition_type='EI', exact_feval=False, maximize=True)

# x_next_eval = bo_step.suggest_next_locations() #Gpy-opt method to suggest next point based
# print(x_next_eval)


In [ ]:
iter_count = 2 #to keep the count of number of iterations
current_iter = 0
# make a copy of the array containing the initial dataset, just to be safe and
# distinguish the initial data from the one we would be appending
X_updating = X_init
Y_updating = Y_init

In [ ]:
while current_iter<iter_count:
    #the following line creates the BO object to run the optimisation
    #Few points to note:
    # 1. f is set to 'None' since we are evaluating our objective externally
    # 2. model type is ‘GP’, standard Gaussian process.
    # 3. acqusition function used is 'EI', Expected Improvement (this did not ask me to tune psi, would have to look into this)
    # 4. I have set exact_eval to 'False', assuming our output would be noisy and not an exact value
    # 5. maximize is set to 'True', since we want to maximise the output
    bo_step = GPyOpt.methods.BayesianOptimization(f = None, domain = domain, model_type='GP',
                                                  X = X_updating, Y = Y_updating, initial_design_numdata=12,
                                                  acquisition_type='EI', exact_feval=True, maximize=True)

    #list down the errors that we can have when we perform the physical experiment

    x_next_eval = bo_step.suggest_next_locations() #Gpy-opt method to suggest next point based
    print(x_next_eval)
    y_next = 98.257

    # --> the following two lines add the new data entry to our existing dataset
    X_updating = np.vstack((X_updating, x_next_eval))
    Y_updating = np.vstack((Y_updating, y_next))

    current_iter += 1